In [3]:
from langchain_community.llms import ollama
import json

# Define the system prompt
SYSTEM_PROMPT = """
Generate a JSON object where:
1. "input" is a single string where a user requests information about hotels in a specific location. Additional preferences or requirements (such as star ratings or amenities) may or may not be mentioned. The input is provided as plain text, not JSON.
2. "output" contains the corresponding Amadeus API call in JSON format:
   - The "action" field is always ["search_hotels"].
   - The "action_input" field includes:
     - "cityCode": A 3-letter city code for the requested location.
     - Optional fields to include only if explicitly mentioned in the user request:
       - "ratings": An integer representing the hotel ratings (out of 5), included only if ratings are mentioned.
       - "amenities": A list of amenities required by the user, included only if amenities are mentioned. Supported values: SWIMMING_POOL, SPA, FITNESS_CENTER, AIR_CONDITIONING, PETS_ALLOWED, WIFI.
   - If ratings or amenities are not mentioned, these fields should be excluded from the output entirely (not set to empty or null).

3. Generate one example per response object, with variations in the input phrasing and combinations of optional fields. Ensure no hallucination of fields or values that are not explicitly requested in the "input."
"""


# Initialize the Ollama LLM
llm = ollama.Ollama(model="llama3.2", format="json", verbose=False)

# Generate 10 examples with varying temperatures
examples = []
for i in range(5):
    # Update temperature for diversity
    temperature = 1

    # Generate a response using the fixed system prompt
    response = llm.invoke(
        SYSTEM_PROMPT,
        config={"temperature": temperature}
    )
    
    try:
        # Parse the response and append it to the examples list
        example = json.loads(response)
        examples.append(example)
    except json.JSONDecodeError:
        print(f"Failed to decode response at iteration {i + 1}: {response}")

# Print all generated examples
for idx, example in enumerate(examples):
    print(f"Example {idx + 1}:")
    print(json.dumps(example, indent=4))

ConnectionError: HTTPConnectionPool(host='localhost', port=11434): Max retries exceeded with url: /api/generate (Caused by NewConnectionError('<urllib3.connection.HTTPConnection object at 0x00000215BD762D10>: Failed to establish a new connection: [WinError 10061] No connection could be made because the target machine actively refused it'))

In [19]:
from ollama import chat
from ollama import ChatResponse
import json

# Define the system prompt to guide the model's behavior
system_prompt = {
    'role': 'system',
    'content': (
        """You are a json creater.
        Your output will be in the following format    
        {"input": "Can you find me some hotels in paris with a swimming pool and WiFi?", "output": {"action": ["search_hotels"],"action_input": {"amenities": ["SWIMMING_POOL","WIFI"],"destination": "PAR"}}}

        Give both input fields and output fields as a response
        don't respond like a respondant
        don't do line breaking with '\n'
        just give me the pure data.
        Output only one example
        """
    )
}

# Example user prompt (replace with actual user input)
# user_prompt = "Why is the sky blue?"
user_prompt = """
1. "input" is a single string where a user requests information about hotels in a specific location. Additional preferences or requirements (such as star ratings or amenities) may or may not be mentioned. The input is provided as plain text, not JSON.
2. "output" contains the corresponding Amadeus API call in JSON format:
   - The "action" field is always ["search_hotels"].
   - The "action_input" field includes:
     - "cityCode": A 3-letter city code for the requested location.
     - Optional fields to include only if explicitly mentioned in the user request:
       - "ratings": An integer representing the hotel ratings (out of 5), included only if ratings are mentioned.
       - "amenities": A list of amenities required by the user, included only if amenities are mentioned. Supported values: SWIMMING_POOL, SPA, FITNESS_CENTER, AIR_CONDITIONING, PETS_ALLOWED, WIFI.
   - If ratings or amenities are not mentioned, these fields should be excluded from the output entirely (not set to empty or null).

3. Use different cities in Asia
4. You can exclude or include different amenities and ratings
5. Make the input wording interesting
6. Only give one response
"""
empty = []

for i in range(100):
    # Call the chat model with both the system prompt and the user prompt

    try:
        validation_response : ChatResponse = chat(model='llama3.2', messages=[
            system_prompt,
            {'role': 'user', 'content': user_prompt}],
            stream=False,
            options={"temperature":0.6}
        )

        validation_output = validation_response['message']['content']
        response_json = json.loads(validation_output)

        empty.append(response_json)
    except:
        continue

In [20]:
empty

[{'input': 'Can you find me some luxurious hotels in Tokyo with a spa, air conditioning, and WiFi? With an average rating of 4 out of 5 stars?',
  'output': {'action': ['search_hotels'],
   'action_input': {'cityCode': 'TKO',
    'ratings': 4,
    'amenities': ['SPRINGA', 'AIR_CONDITIONING', 'WIFI']}}},
 {'input': 'Can you find me some luxurious hotels in Tokyo with a spa, air conditioning, and free WiFi, rating 4 out of 5 stars?',
  'output': {'action': ['search_hotels'],
   'action_input': {'cityCode': 'TKO',
    'ratings': 4,
    'amenities': ['SPA', 'AIR_CONDITIONING', 'WIFI']}}},
 {'input': 'Can you find me some luxury hotels in Tokyo with a spa, fitness center, and WiFi that are 5-star rated?',
  'output': {'action': ['search_hotels'],
   'action_input': {'cityCode': 'TKO',
    'ratings': 5,
    'amenities': ['SPA', 'FITNESS_CENTER', 'WIFI']}}},
 {'input': 'Can you recommend some luxurious hotels in Bangkok with a spa, fitness center, and WiFi for business travelers?',
  'output'

In [57]:
import json
response_json = json.loads(test)